# 使用 LlamaIndex 搭建本地 RAG 应用

https://github.com/datawhalechina/handy-ollama/tree/main/notebook/C7/LlamaIndex_RAG

本文档将详细介绍如何使用 LlamaIndex 框架来搭建本地 RAG（Retrieval-Augmented Generation）应用。  
通过集成 LlamaIndex，可以在本地环境中构建一个 RAG 系统，结合检索与生成的能力，以提高信息检索的效率和生成内容的相关性。  
可以自定义本地知识库路径，通过 LlamaIndex 构建索引，然后利用索引进行上下文对话。



## 模型下载

本例中使用的是 llama3.1 模型，可以根据自身电脑配置，使用合适的模型。


https://github.com/ollama/ollama


```bash
brew install ollama
ollama pull llama3.2
ollama pull nomic-embed-text

conda activate llm-study
pip install llama-index-llms-ollama
pip install llama-index-embeddings-ollama
pip install llama-index-readers-file
```


## 加载数据&构建索引

加载当前目录下 data 文件夹中所有的文档，并加载到内存中。

- Settings.embed_model ： 全局的 embed_model 属性。示例代码中将创建好的嵌入模型赋值给全局的 embed_model 属性；
- Settings.llm ： 全局的 llm 属性。示例代码中将创建好的语言模型赋值给全局的 llm 属性；
- VectorStoreIndex.from_documents：使用之前加载的文档构建索引，并转换成向量，便于快速检索。

通过 Settings 全局属性的设置，在后面的索引构建以及查询的过程中就会默认使用相应的模型。

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

documents = SimpleDirectoryReader("data").load_data()

# nomic embedding model
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# ollama
Settings.llm = Ollama(model="llama3.2", request_timeout=360.0)

index = VectorStoreIndex.from_documents(
    documents,
)


## 查询数据

index.as_query_engine()：根据之前构建好的索引，创建查询引擎。该查询引擎可以接收查询，返回检索后的响应。


In [4]:
# 查询数据
query_engine = index.as_query_engine()
response = query_engine.query("Datawhale是什么?")
print(response)

Datawhale是一个开源组织。


## 检索上下文进行对话

由于检索到的上下文可能会占用大量可用的 LLM 上下文，因此需要为聊天历史记录配置较小 token 限制！

chat_mode 可以根据使用场景，选择合适的模式，支持的模式如下：

- best（默认）：使用带有查询引擎工具的代理（react 或 openai）；
- context：使用检索器获取上下文；
- condense_question：将问题进行浓缩；
- condense_plus_context：将问题进行浓缩并使用检索器获取上下文；
- simple：直接使用 LLM 的简单聊天引擎；
- react：使用带有查询引擎工具的 react 代理；
- openai：使用带有查询引擎工具的 openai 代理。


In [5]:
# 检索上下文进行对话
from llama_index.core.memory import ChatMemoryBuffer
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions."
    ),
)

response = chat_engine.chat("Datawhale是什么？")
print(response)

Datawhale是一个专注于数据科学与 AI 领域的开源组织，成立于2018年。它聚集了来自多个领域和企业的优秀学习者和团队成员，致力于构建一个开放、包容的社区，为学习者提供成长和发展的机会。Datawhale的使命是 "for the learner, and learners together."


In [6]:
response = chat_engine.chat("Datawhale是哪一年成立的？")
print(response)

Datawhale在2018年成立。


In [7]:
response = chat_engine.chat("Datawhale目标是什么？")
print(response)

Datawhale的目标是"for the learner，和学习者一起成长。"这意味着它关心的是学习者的成长和发展，它们希望通过开源社区、学习资源和开放性探索来帮助学习者在数据科学和 AI 领域取得成就。


## 向量索引的存储和加载

storage_context.persist 存储向量索引。
load_index_from_storage 加载向量索引。

In [8]:
# 存储向量索引
persist_dir = 'data/'
index.storage_context.persist(persist_dir=persist_dir)

# 加载向量索引
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index= load_index_from_storage(storage_context)


## streamlit 应用

```
pip install streamlit
pip install llama_index
pip install watchdog

streamlit run ollama-llamaindex-app.py
```

In [ ]:
import streamlit as st
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.memory import ChatMemoryBuffer
import os
import tempfile
import hashlib

# OLLAMA_NUM_PARALLEL：同时处理单个模型的多个请求
# OLLAMA_MAX_LOADED_MODELS：同时加载多个模型
os.environ['OLLAMA_NUM_PARALLEL'] = '2'
os.environ['OLLAMA_MAX_LOADED_MODELS'] = '2'


# Function to handle file upload
def handle_file_upload(uploaded_files):
    if uploaded_files:
        temp_dir = tempfile.mkdtemp()
        for uploaded_file in uploaded_files:
            file_path = os.path.join(temp_dir, uploaded_file.name)
            with open(file_path, "wb") as f:
                f.write(uploaded_file.getvalue())
        return temp_dir
    return None


# Function to calculate a hash for the uploaded files
def get_files_hash(files):
    hash_md5 = hashlib.md5()
    for file in files:
        file_bytes = file.read()
        hash_md5.update(file_bytes)
    return hash_md5.hexdigest()


# Function to prepare generation configuration
def prepare_generation_config():
    with st.sidebar:
        st.sidebar.header("Parameters")
        max_length = st.slider('Max Length', min_value=8, max_value=5080, value=4056)
        temperature = st.slider('Temperature', 0.0, 1.0, 0.7, step=0.01)
        st.button('Clear Chat History', on_click=clear_chat_history)

    generation_config = {
        'num_ctx': max_length,
        'temperature': temperature
    }
    return generation_config


# Function to clear chat history
def clear_chat_history():
    st.session_state.messages = [{"role": "assistant", "content": "你好，我是你的助手，你需要什么帮助吗？"}]


# File upload in the sidebar
st.sidebar.header("Upload Data")
uploaded_files = st.sidebar.file_uploader("Upload your data files:", type=["txt", "pdf", "docx"],
                                          accept_multiple_files=True)

generation_config = prepare_generation_config()


# Function to initialize models
@st.cache_resource
def init_models():
    embed_model = OllamaEmbedding(model_name="nomic-embed-text")
    Settings.embed_model = embed_model

    llm = Ollama(model="llama3.2", request_timeout=360.0,
                 num_ctx=generation_config['num_ctx'],
                 temperature=generation_config['temperature'])
    Settings.llm = llm

    documents = SimpleDirectoryReader(st.session_state['temp_dir']).load_data()
    index = VectorStoreIndex.from_documents(documents)

    memory = ChatMemoryBuffer.from_defaults(token_limit=4000)
    chat_engine = index.as_chat_engine(
        chat_mode="context",
        memory=memory,
        system_prompt="You are a chatbot, able to have normal interactions.",
    )

    return chat_engine


# Streamlit application
st.title("💻 Local RAG Chatbot 🤖")
st.caption("🚀 A RAG chatbot powered by LlamaIndex and Ollama 🦙.")

# Initialize hash for the current uploaded files
current_files_hash = get_files_hash(uploaded_files) if uploaded_files else None


# Detect if files have changed and init models
if 'files_hash' in st.session_state:
    if st.session_state['files_hash'] != current_files_hash:
        st.session_state['files_hash'] = current_files_hash
        if 'chat_engine' in st.session_state:
            del st.session_state['chat_engine']
            st.cache_resource.clear()
        if uploaded_files:
            st.session_state['temp_dir'] = handle_file_upload(uploaded_files)
            st.sidebar.success("Files uploaded successfully.")
            if 'chat_engine' not in st.session_state:
                st.session_state['chat_engine'] = init_models()
        else:
            st.sidebar.error("No uploaded files.")
else:
    if uploaded_files:
        st.session_state['files_hash'] = current_files_hash
        st.session_state['temp_dir'] = handle_file_upload(uploaded_files)
        st.sidebar.success("Files uploaded successfully.")
        if 'chat_engine' not in st.session_state:
            st.session_state['chat_engine'] = init_models()
    else:
        st.sidebar.error("No uploaded files.")
        

# Initialize chat history
if 'messages' not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "你好，我是你的助手，你需要什么帮助吗？"}]

# Display chat messages from history
for message in st.session_state.messages:
    with st.chat_message(message['role'], avatar=message.get('avatar')):
        st.markdown(message['content'])

# Display chat input field at the bottom
if prompt := st.chat_input("Ask a question about Datawhale:"):

    with st.chat_message('user'):
        st.markdown(prompt)

    # Generate response
    response = st.session_state['chat_engine'].stream_chat(prompt)
    with st.chat_message('assistant'):
        message_placeholder = st.empty()
        res = ''
        for token in response.response_gen:
            res += token
            message_placeholder.markdown(res + '▌')
        message_placeholder.markdown(res)

    # Add messages to history
    st.session_state.messages.append({
        'role': 'user',
        'content': prompt,
    })
    st.session_state.messages.append({
        'role': 'assistant',
        'content': response,
    })

2025-03-22 15:03:16.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 15:03:16.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 15:03:16.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 15:03:16.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 15:03:16.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 15:03:16.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 15:03:16.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 15:03:16.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar